<a href="https://colab.research.google.com/github/Trrunghau/AI/blob/main/Plant_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.models import load_model
from os import listdir
from numpy import asarray,save
from keras.utils import load_img, img_to_array

In [20]:
### thay doi ti le anh tu 0->255 thanh 0->1
train = ImageDataGenerator(rescale =1.0/255.0,rotation_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   zoom_range = 0.2)
test = ImageDataGenerator(rescale=1.0/255.0)

In [21]:
train_data = train.flow_from_directory('/content/drive/MyDrive/data', target_size=(128,128),batch_size = 10, class_mode='categorical')
test_data = test.flow_from_directory('/content/drive/MyDrive/data_test', target_size=(128,128), class_mode='categorical')

Found 1190 images belonging to 6 classes.
Found 219 images belonging to 6 classes.


In [22]:
print('train_index :', train_data.class_indices)
print('test_index:', test_data.class_indices)

train_index : {'Corn_Common_Rust': 0, 'Corn_Gray_Leaf_Spot': 1, 'Corn_Healthy': 2, 'bean_healthy': 3, 'bean_leaf_spot': 4, 'bean_rust': 5}
test_index: {'Corn_Common_Rust': 0, 'Corn_Gray_leaf_spot': 1, 'Corn_Heathy': 2, 'bean_healthy': 3, 'bean_leaf_spot': 4, 'bean_rust': 5}


In [23]:
model= Sequential()
## lop CNN1
model.add(Conv2D(128, kernel_size=(3,3), activation='relu',input_shape=(128,128,3), padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
## lop CNN2
model.add(Conv2D(64,(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
## lop CNN3
model.add(Conv2D(32,(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
## lop CNN4
model.add(Conv2D(16,(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
## lop CNN5
model.add(Conv2D(8,(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))

classes = 6

In [24]:
# chuyen du lieu tu CNN ve ANN ( 2 chieu ve 1 chieu)
model.add(Flatten())
##lop hidden1
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
# lop outpu
model.add(Dense(classes, activation='softmax'))

In [25]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 128, 128, 128)     3584      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 64, 64, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 64)        73792     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        18464     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 32)      

In [ ]:
history=model.fit(train_data,epochs=80,batch_size=256,verbose=1, 
                  validation_data=test_data)

Epoch 1/80
119/119 [==============================] - 146s 1s/step - loss: 1.6112 - accuracy: 0.2723 - val_loss: 1.7699 - val_accuracy: 0.2740
Epoch 2/80
119/119 [==============================] - 143s 1s/step - loss: 1.4649 - accuracy: 0.3395 - val_loss: 1.7750 - val_accuracy: 0.1781
Epoch 3/80
119/119 [==============================] - 142s 1s/step - loss: 1.4731 - accuracy: 0.3294 - val_loss: 1.7473 - val_accuracy: 0.2922
Epoch 4/80
119/119 [==============================] - 142s 1s/step - loss: 1.4177 - accuracy: 0.3824 - val_loss: 1.5748 - val_accuracy: 0.3699
Epoch 5/80
119/119 [==============================] - 140s 1s/step - loss: 1.1180 - accuracy: 0.5765 - val_loss: 1.1801 - val_accuracy: 0.5342
Epoch 6/80
119/119 [==============================] - 140s 1s/step - loss: 0.9297 - accuracy: 0.6420 - val_loss: 1.0835 - val_accuracy: 0.5799
Epoch 7/80
119/119 [==============================] - 142s 1s/step - loss: 0.8481 - accuracy: 0.6714 - val_loss: 0.9938 - val_accuracy: 0.6119

In [ ]:
value=model.evaluate(test_data,verbose=0)
print('loss', value[0])
print('accuracy', value[1])

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/modelfinal5.h5')

In [ ]:

accuracy =history.history['accuracy']
loss = history.history['loss']
epochs=range(len(accuracy))

import matplotlib.pyplot as plt
plt.plot(epochs, accuracy, 'bo', label='Training accuracy', color='green')
plt.plot(epochs,loss , 'b', label ='Training loss')
plt.show()

In [ ]:
## lay model tu google drive va tien hanh nhan dien
model = load_model('/content/drive/MyDrive/Colab Notebooks/modelfinal5.h5')
name_foods=['Corn_Common_Rust', 'Corn_Gray_Leaf_Spot', 'Corn_Healthy', 'bean_healthy', 'bean_leaf_spot', 'bean_rust']
filePath = '/content/drive/MyDrive/data_test/Corn_Gray_leaf_spot/Corn_Gray_Spot (501).JPG'
testImage = load_img(filePath, color_mode = 'rgb', target_size=(128,128))
arrayImage = img_to_array(testImage).reshape(1,128,128,3)/255
pred = int(np.argmax(model.predict(arrayImage)))
print("Predicted: ", name_foods[pred])
plt.imshow(testImage)